# Converter

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nelson_lab_to_nwb.intan_sessions_converter import IntanSessionNWBConverter
from neuroconv.utils.dict import load_dict_from_file, dict_deep_update
from pynwb import NWBHDF5IO, NWBFile

In [3]:
# set the source data for the Converter
source_data = dict(
    IntanMultifilesRaw=dict(
        folder_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954"
    ),
    AIMScore=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx",
        timestamp_column_name="Time (minutes relative to injection)",
        aims_column_name="AIMS",
    )
)

# Instantiate the Converter
converter = IntanSessionNWBConverter(source_data)

# Automatically fetch metadata from files, then update it with user-defined metadata
source_metadata = converter.get_metadata()
user_metadata_file = "/mnt/shared_storage/Github/nelson-lab-to-nwb/src/nelson_lab_to_nwb/intan_sessions_converter/metadata_example.yaml"
user_metadata = load_dict_from_file(file_path=user_metadata_file)
metadata = dict_deep_update(source_metadata, user_metadata)

Source data is valid!


In [4]:
# Conversion options
conversion_options = dict(
    IntanMultifilesRaw=dict(
        stub_test=True,
    )
)

# Run conversion
converter.run_conversion(
    metadata=metadata,
    nwbfile_path="converted_intan_session.nwb",
    overwrite=True,
    conversion_options=conversion_options
)

Metadata is valid!
conversion_options is valid!


/home/luiz/anaconda3/envs/nelson_lab_to_nwb_env/lib/python3.12/site-packages/pynwb/file.py:146: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)
/home/luiz/anaconda3/envs/nelson_lab_to_nwb_env/lib/python3.12/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'TimeSeries/timestamps': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


NWB file saved at converted_intan_session.nwb!


In [5]:
io = NWBHDF5IO("converted_intan_session.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

,location,group,group_name,channel_name
id,,,,
0,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-000
1,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-001
2,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-002
3,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-003
,location,group,group_name,channel_name
id,,,,
0,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-000
1,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-001
2,unknown,B pynwb.ecephys.ElectrodeGroup at 0x140549401784928\nFields:\n description: Group B electrodes.\n device: Intan pynwb.device.Device at 0x140556517416576\nFields:\n description: Intan recording\n manufacturer: Intan\n\n,B,B-002


# Multiple Intan files

ref: https://github.com/SpikeInterface/spikeinterface/issues/2620

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nelson_lab_to_nwb.interfaces.intan_multi_interface import IntanMultifilesRecordingInterface
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO, NWBFile

In [ ]:
folder_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954"
interface = IntanMultifilesRecordingInterface(folder_path=folder_path, verbose=False)

# Extract what metadata we can from the source files
metadata = interface.get_metadata()
# session_start_time is required for conversion. If it cannot be inferred
# automatically from the source files you must supply one.
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata["NWBFile"].update(session_start_time=session_start_time)

In [ ]:
nwbfile_path = "intan_data.nwb"
interface.run_conversion(
    nwbfile_path=nwbfile_path,
    metadata=metadata,
    overwrite=True
)

In [ ]:
io = NWBHDF5IO("intan_data.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

# AIM Score

In [ ]:
from nelson_lab_to_nwb.interfaces import AIMScoreInterface

In [ ]:
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx"
interface = AIMScoreInterface(
    file_path=file_path,
    timestamp_column_name="Time (minutes relative to injection)",
    aims_column_name="AIMS",
    verbose=True
)

In [ ]:
interface.df